Realizado por: Sergio Duque Mejía

## $Algoritmo$ $de$ $Metropolis$
Los eventos creados son los estados $\phi$. 

1. Se lanza un estado inicial $\phi_{0}$.
2. Generamos aleatoriamente otro vector de estado $\phi^{'}$
3. Se calcula: $\Delta S = -ln \left \{ \frac{P(\phi^{'})}{P(\phi_{0})} \right \}=-ln \{ \alpha \}$.
4. Si $\Delta S < 0$ entonces $\phi^{'} = \phi_{1}$.
5. Si $\Delta S > 0$ entonces sea $r\equiv numero$ $aleatorio$ que se compara con con $\alpha$ es decir: 
  1. Si $\alpha > r$ entonces $\phi^{'}=\phi_{1}$
  2. Si no ocurre lo anterior, se descarta $\phi^{'}$.
6. Se repite el proceso, vuelve a $\phi_{2}$ y se genera otro vector aleatorio $\phi^{'}$

Si se repite N veces, se genera N+1 (si se acepta todos) veces $\phi$'s, estos datos son la nueva muestra. Este algoritmo sirve para ver si los datos del experimento se ajustan adecuadamente a la PDF. 



In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from sklearn.metrics import mean_squared_error

In [ ]:
!apt-get install git dpkg-dev cmake g++ gcc binutils libx11-dev libxpm-dev libxft-dev libxext-dev tar gfortran subversion

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
!ls drive/'My Drive'/Montecarlo/

In [ ]:
!tar -xf drive/"My Drive"/Montecarlo/root_build_google_yikes.tar

In [ ]:
!ls -ltrh root_build/

In [ ]:
import sys
sys.path.append("/content/root_build/")
sys.path.append("/content/root_build/bin/")
sys.path.append("/content/root_build/include/")
sys.path.append("/content/root_build/lib/")

In [ ]:
import ROOT

In [ ]:
DataChain=ROOT.TChain("SystemTree")
DataChain.Add("/content/drive/My Drive/Montecarlo/SergioDuque.root")
NumberOfEventsToCheck=DataChain.GetEntries()
print("We are going to analyze ",NumberOfEventsToCheck, " events")

In [ ]:
State = ROOT.TH1F( 'State', "Montecarlo's System", 200, 100, 400 )
Old_events = []
for entry in DataChain:
  Old_events.append(entry.x)

for i in range(len(Old_events)):
  State.Fill(Old_events[i])

In [ ]:
Bg = ROOT.TCanvas( 'Bg', 'Background' )
Bg.SetFillColor( 40 )

State.SetFillColor( 45 )
State.Draw()

Bg.Update()

ROOT.gROOT.GetFunction("State")
ROOT.gROOT.GetListOfCanvases().Draw()

In [ ]:
# Funcion de distribucion de Landau
def ft(t,x,mu,c):
  return np.exp(-t) * np.cos((t*((x-mu)/c))+((2*t/np.pi)*np.log(t/c)))/(np.pi*c)

def Pdf_landau(x,mu,c):
  return quad(ft, 0, np.inf, args=(x, mu, c))[0]

# Algoritmo de Metropolis-Hastings
def Metropolis(lim,pdf,N,phi_0,mu,c):
  sample = [phi_0]
  i = 0
  while len(sample) < N:
    phi_prime =  np.random.uniform(low = lim[0],high = lim[1])
    Delta_S = -np.log(pdf(phi_prime,mu,c)/pdf(sample[0],mu,c))
    if Delta_S < 0:
      sample.append(phi_prime)
    elif Delta_S > 0:
      r = np.random.random()
      alpha = pdf(phi_prime,mu,c)/pdf(sample[0],mu,c)
      if r < alpha: 
        sample.append(phi_prime)
      elif r > alpha: 
        pass
  return sample

### $Prueba$
Se verifica el algoritmo de Metropolis-Hastings para una pdf Landau con mu=0, c=1 en el intervalo de (-1,1)

In [ ]:
#Genero Datos de prueba de la pdf Landau con mu=0 y c=1 definiendo un x entre (-1,1)
prueba = []
xdt = np.linspace(-1,1,100)
x_v = []
for i in xdt:
  a = Pdf_landau(i,0,0.1)
  prueba.append(a)
  x_v.append(i)

In [ ]:
metric = Metropolis([-1,1],Pdf_landau,1000,-0.2,0,0.1)
plt.plot(x_v,prueba)
plt.hist(metric,30,density=True, facecolor='blue', alpha=0.2)
plt.grid()
plt.show()

### $Generando$ $las$ $muestras$ $y$ $graficas$

In [ ]:
# Extraccion de los datos experimentales de ROOT a un histograma de Matplotlib
exp_d = np.array(Old_events)
hist_exp = plt.hist(exp_d , 100,density=True, facecolor='blue', alpha=0.5,range=(50,800))
plt.grid()
plt.title('Histograma del experimento')
plt.show()

In [ ]:
# Funcion de Minimizacion de Chi cuadrado para dos parametros, integrado el algoritmo de metropolis
def Chisquare(a,b):
    for i in a:
        for j in b:
          hist_sample = np.array(Metropolis([50,800],Pdf_landau,200,float(i),i,j))
          hist = np.histogram(hist_sample,normed=True,bins=200,range=(50,800))
          chi_square = mean_squared_error(hist_exp,hist)
          if i == a[0] and j == b[0]:
            parametros = [i,j]
            chi = chi_square
          elif chi > chi_square:
            parametros = [i,j]
            chi = chi_square
    return chi,parametros

In [ ]:
# Obtencion de Parametros y valor de Chi cuadrado
a = np.arange(100,150,3)
b = np.arange(10,20,3)

chi,parametros = Chisquare(a,b)
print(parametros)
print(chi)

In [ ]:
# Grafica de resultados usando los parametros encontrados
metr = Metropolis([50,800],Pdf_landau,1000,130,130,10)
with plt.style.context('Solarize_Light2'):
  plt.hist(exp_d, 200,density=True, color="blue", histtype='step', alpha=1,range=(50,800),label="Experimentales")
  plt.hist(np.array(metr), 50,density =True, facecolor='green', alpha=0.5, label="Generados por M.H")
  plt.legend()
  plt.show()

In [ ]:
# Analisis de c: 
rango = [5,10,20,35]
with plt.style.context('Solarize_Light2'):
    plt.hist(exp_d, 200,density=True, color="blue", histtype='step', alpha=1,range=(50,800),label="Experimentales")
    for i in rango:
      Mc  =  Metropolis([50,800],Pdf_landau,1000,130,130,i)
      plt.hist(np.array(Mc), 50,density =True, alpha=0.5,label= r'con $c=$'+str(i))
      plt.legend()
plt.show() 